In [ ]:
# 3rd part of the project, write a bunch of functions to completely script the code

In [1]:
# import libraries
# conda install psycopg2
# pip install psycopg2
# pip install mysql-connector-python

import os # interaction with operating system
import numpy as np # to work with arrays
import pandas as pd # multi-dimentional arrays
import psycopg2 # postgres wrapper, make a connection to database
#import mysql.connector

In [2]:
# isolate only the CSV files
# write a for loop to check each item and only retrive those that is a csv

def csv_files():
    
    csv_files = [] # initialize empty file list
    for file in os.listdir(os.getcwd()):# kinda the same as ls but now its saved in a list
    
        if file.endswith('.csv'):
        
            csv_files.append(file)
            
    return csv_files

In [3]:
csv_files()

['us_cities.csv',
 'average-latitude-longitude-countries.csv',
 'world_cities.csv']

In [4]:
# make a new directory

def configure_dataset_directory(csv_files, dataset_dir):

    # create the bash command to make a new directory
    # mkdir dataset_dir

    # if the folder already exist, ignore
    # if you don't have sudo permission to make a directory, it won't work
    try:

        mkdir = 'mkdir {0}'.format(dataset_dir) #
        os.system(mkdir)
    except:
        print('File aleady created')
        
    # move the CSV files in the new directory

    # mv filename directory
    for csv in csv_files:

        mv_file = "mv '{0}' {1}".format(csv, dataset_dir) # the qoutes here help python to read the file name as a single file incase there's afile with a space
        os.system(mv_file)
        print(mv_file)
    return

In [5]:
# autmate the file reading process and also be able to handle multiple csv's
# replace manual typing of csv's names with the path and file name
# create a dictionary to hold all the dataframes
# define the path to the csv file
# for loop to get the file name, read each as a panda dataframe
def create_df(dataset_dir, csv_files):
    
    # path to the csv files
    data_path = os.getcwd() + '/' + dataset_dir + '/'
    
    # loop through the files and create the dataframe
    data = {}
    for file in csv_files:

        try:
            data[file] = pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            data[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
        
    return data

In [6]:
# clean data name if need be
# lower case letter
# remove all white spaces
# replace -, /, \\, $ with _

# write script that will enforce the rules above
# using escape "/" at the end og my code line to go to the next line instead of havinh one very long line of code (clean code)
# using "r" raw string, use the symbol in it's raw

def clean_tbl_name(filename):
    
    clean_tbl_name = filename.lower().replace(" ","_").replace("?","") \
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                    .replace(r")","").replace(r"(","").replace("$","")
    
    # remove .csv extention from clean_tbl_name
    
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    
    return tbl_name

In [7]:
def clean_clm_name(dataframe):
    
    # write list comprehension to clean up names

    dataframe.columns = [x.lower().replace(" ","_").replace("?","") \
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                    .replace(r")","").replace(r"(","").replace("$","") for x in dataframe.columns]
    
    # SQL data types don't match the pandas data types
    # replace the pandas data types with SQL data types, using a dictionary
    
    # because this script can be run with 'any' cvs file - I am going genaral with the replacements - feel free to add
    
    replacements = {
    'object': 'varchar',
    'float64': 'double precision',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
    }
    
    # comma will seperate the lines(in the sql statement, join the comma wit the column name and data type (thats the curly brackets
    # format i to place the column names and data types respectively, for loop to go through each element in the list(data.lolumns and data.dtypes
    # dtypes = panda datatypes, replace those with sql data types by using the replace function using the replacements dictionary 
    # zip everything up, want the column names next to the dtypes names
    
    # table schema

    clm_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    return clm_str, dataframe.columns
    

In [8]:
# create database (dbBeaver)
# Connect to database
# connect_str = "host=jdbc:postgresql://localhost:5432/project_data \
#            dbname='project_data' \
#            user='postgres' pasword='post88'

# establish connection
    
def upload_to_db(host, database, user, password, tbl_name, clm_str, file, dataframe, dataframe_columns):
    
    conn_string = "host=%s database=%s user=%s password=%s" % (host, dbname, user, password)
    connect = psycopg2.connect(conn_string)

    cursor = connect.cursor()

    print("opened DB success")
    # drop tables with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))
    
    # create table
    cursor.execute("create table %s (%s);" % (tbl_name, clm_str))
    print('{0} was created successfully'.format(tbl_name))
    
    #connect.close()
    
    # insert values to table

    #save pandas data-frame to csv file
    dataframe.to_csv(file , header=dataframe_columns, index=False, encoding="utf-8")


    # open csv file, save it as an object, and upload to db
    # taking the csv file, opoen it up in the ram, then preparing it to insert into database table
    mon_file = open(file)
    print("filed opened in memory")
    
    # upload to db

    SQL_STATEMENT = """
    COPY %s from STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """
    
    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=mon_file)
    print("file copied to db")
    
    #grant multiple user access to table to anyone who has access to my database

    cursor.execute("grant select on %s to public" % tbl_name)
    connect.commit()

    cursor.close()
    print("table {0} to db completed".format(tbl_name))
    
    return